In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import *

In [2]:
# read glossary
glossary = pd.read_csv('../data/metadata2kg/round2/r2_glossary_processed.csv')
# read sample metadata
metadata = pd.read_json('../data/metadata2kg/round2/r2_test_metadata.jsonl', lines=True)
# concat index number with id to make it unique
metadata['id'] = metadata.index.astype(str) + '_' + metadata['id']

## Table Completion

In [3]:
prompt = """Given a table '{table_name}' with columns:

{col_descriptions}

Your task is to determine which of the above is the best description for the column '{label}' given the context of the other columns.
I will provide you with {num} descriptions, each indicated by number identifier [].

{descriptions}
You must give only the best description and provide the unique identifier using the output format [].

### Example
Analysis: <your step-by-step reasoning here>

[identifier of the best fitting description]

### Answer
"""

In [4]:
rerank = read_jsonl_file('mapping_completion.jsonl')

In [ ]:
import asyncio
import os
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)
    return wrapped

@background
def fn(chunk):
    for i, table in chunk.iterrows():
      json_file = 'test_completion_table_gpt_sc0/output' + str(i) + '.json'

      # convert string to seed offset
      seed_offset = sum([ord(c) for c in json_file]) 

      # if directory does not exist, create it
      if not os.path.exists(json_file[:json_file.rfind('/')]):
        os.makedirs(json_file[:json_file.rfind('/')])

      if os.path.isfile(json_file):
        continue
      id = '_'.join(table['id'].split('_')[1:])
      maps = [m['mappings'] for m in rerank if m['id'] == id][-1]
      top_k = [int(m['id']) for m in maps][:5]

      document = glossary[glossary['id'].isin(top_k)]
      print(len(document['id'].tolist()), document['id_no_prefix'].tolist())

      print('Table: ', i, table['label'])

      for retry in range(10):
        try:          
            # second prompt
            messages = [
                {"role": "user", "content": prompt.format(table_name=table['table_name'],
                                                        label=table['label'],
                                                        descriptions=print_descriptions((document['label'] + ", " +document['desc']).tolist()),
                                                        num = len(document['desc'].tolist()),
                                                        col_descriptions='\n'.join(get_column_descriptions(table, glossary, rerank))
                                                        )}
            ]
            print(messages[0]['content'])
            m2 = message_gpt(messages, temperature=0.0, seed=retry+seed_offset)
            print(m2)

            # write output to json file
            with open(json_file, 'w') as f:
                # write json object
                f.write(json.dumps({
                    'table': table.to_json(),
                    'document': document['id_no_prefix'].tolist(),
                    'analysis_1_match': m2,
                    'k_matches': document['id_no_prefix'].tolist(),
                    '1_match': [document.iloc[extract_identifiers(m2)[-1]]['id_no_prefix']],
                }, indent=4))
               
            # for now never retry
            break
        except Exception as e:
            print(e)
            pass

        print('Retrying: ', json_file)
        time.sleep(2)

n_clients = 20

for i in range(n_clients):
    fn(metadata[metadata.index%n_clients==i])
    #break

## Merge

In [5]:
# self consistency join
import glob
import os
import json

folders = ['test_completion_table_gpt_sc0']
output_folder = 'test_completion_table'

# if directory exists, its contents
if os.path.exists(output_folder):
    for file in glob.glob(output_folder + '/*.json'):
        os.remove(file)
else:
    os.makedirs(output_folder)
            
for folder in folders:
    for file in glob.glob(folder + '/*.json'):
        with open(file, 'r') as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(file)
                continue

        # if file does not exist, write it and copy
        if not os.path.isfile(output_folder + '/' + os.path.basename(file)):
            # write output to json file
            with open(output_folder + '/' + os.path.basename(file), 'w') as f:
                # convert k_matches to [{id: id, count: count}]
                data['n_samples'] = 1
                data['k-matches'] = [{'id': id, 'count': 1, 'rrf': 1/(rank+0.01)} for rank, id in enumerate(data['k_matches'])]
                data['1_match'] = [{'id': id, 'count': 1, 'rrf': 1/(rank+0.01)} for rank, id in enumerate(data['1_match'])]
                # write json object
                f.write(json.dumps(data, indent=4))
        # count matches
        else:
            with open(output_folder + '/' + os.path.basename(file), 'r') as f:
                data2 = json.load(f)

            # if k_matches in k-matches, increment count else add to k-matches
            for rank, id in enumerate(data['k_matches']):
                for d in data2['k-matches']:
                    if id == d['id']:
                        d['count'] += 1
                        d['rrf'] += 1/(rank+0.01)
                        break
                else:
                    data2['k-matches'].append({'id': id, 'count': 1, 'rrf': 1/(rank+0.01)})

            # if 1_match in 1_match, increment count else add to 1_match
            for rank, id in enumerate(data['1_match']):
                for d in data2['1_match']:
                    if id == d['id']:
                        d['count'] += 1
                        d['rrf'] += 1/(rank+0.01)
                        break
                else:
                    data2['1_match'].append({'id': id, 'count': 1, 'rrf': 1/(rank+0.01)})

            # sort by count
            data2['n_samples'] = data2['n_samples'] + 1
            data2['k-matches'] = sorted(data2['k-matches'], key=lambda x: x['rrf'], reverse=True)
            data2['1_match'] = sorted(data2['1_match'], key=lambda x: x['rrf'], reverse=True)

            # drop
            data2.pop('analysis_k_matches', None)
            data2.pop('analysis_1_match', None)

            # write output to json file
            with open(output_folder + '/' + os.path.basename(file), 'w') as f:
                # write json object
                f.write(json.dumps(data2, indent=4))

## Eval

In [ ]:
import glob
import json

folder = 'test_completion_table'

files = glob.glob(folder + '/output*.json')

# read all json files
data = []
for file in files:
    with open(file) as f:
        data.append(json.load(f))

# create mapping file
mapping = []
for d in data:
    try:
        # parse table as json object and get id
        table = json.loads(d['table'])
        id = table['id']
        mappings = []

        for matches in d['k-matches']:
            mappings.append({'id': matches['id'], 'score': matches['rrf']})

        # add small value to put best match on top
        best_match = d['1_match'][0]
        for match in mappings:
            if match['id'] == best_match['id']:
                match['score'] += best_match['rrf']

        # sort by score
        mappings = sorted(mappings, key = lambda i: i['score'], reverse=True)

        # round to 3 decimals
        for match in mappings:
            match['score'] = round(match['score'], 3)

        mapping.append({'id': '_'.join(id.split('_')[1:]), 'mappings': mappings})
    except Exception as e:
        print(e)

# change id_no_prefix to id
for table in mapping:
    for match in table['mappings']:
        match['id'] = str(glossary[glossary['id_no_prefix'] == match['id']]['id'].values[0])

# write mapping file
with open('mapping_completion_table.jsonl', 'w') as f:
    for m in mapping:
        f.write(json.dumps(m) + '\n')